[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/whylabs/LanguageToolkit/blob/main/langkit/examples/Sentiment_and_Toxicity.ipynb)

## Install LangKit

First let's install __LangKit__.

In [ ]:
%pip install langkit[all]

# Tracking Sentiment and Toxicity Scores in Text with Langkit

In this example, we'll show how you can easily track sentiment and toxicity scores in text with Langkit.



As an example, we'll use the [tweet_eval dataset](https://huggingface.co/datasets/tweet_eval). We'll use the `hateful` subset of the dataset, which contains tweets labeled as hateful or not hateful.

In [1]:
from datasets import load_dataset

hateful_comments = load_dataset('tweet_eval','hate',split="train", streaming=True)
comments = iter(hateful_comments)


## Initializing the Metrics

To initialize the `toxicity` and `sentiment` metrics, we simply import the respective modules from `langkit`. This will automatically register the metrics, so we can start using them right away by creating a schema by calling `generate_udf_schema`. We will pass that schema to whylogs, so that it knows which metrics to track.

In [2]:
from whylogs.experimental.core.udf_schema import udf_schema
from langkit import toxicity, sentiment

text_schema = udf_schema()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /home/jamie/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


## Profiling the Data

Now we're set to log our data.

To make sure the metrics make sense, we will profile two separate groups of data:
- hateful comments: comments that are labeled as hateful
- non-hateful comments: comments that are labeled as non-hateful

We can expect hateful comments to have a higher toxicity score and a lower sentiment score than non-hateful comments.

Let's see if our metrics will reflect that.

In [4]:
import whylogs as why

# Just initializing the profiles with generic comments.
non_hateful_profile = why.log({"prompt":"I love flowers."}, schema=text_schema).profile()
hateful_profile = why.log({"prompt":"I hate biscuits."}, schema=text_schema).profile()

for _ in range(200):
  comment = next(comments)
  if comment['label'] == 0:
    non_hateful_profile.track({"prompt":comment['text']})
  else:
    hateful_profile.track({"prompt":comment['text']})


Now that we have our profiles, let's check out the metrics. Let's compare the mean for our sentiment and toxicity scores, for each group (hateful and non-hateful):

In [5]:
hateful_sentiment = hateful_profile.view().get_column("prompt.sentiment_nltk").to_summary_dict()["distribution/mean"]
non_hateful_sentiment = non_hateful_profile.view().get_column("prompt.sentiment_nltk").to_summary_dict()["distribution/mean"]

hateful_toxicity = hateful_profile.view().get_column("prompt.toxicity").to_summary_dict()["distribution/mean"]
non_hateful_toxicity = non_hateful_profile.view().get_column("prompt.toxicity").to_summary_dict()["distribution/mean"]

print("######### Sentiment #########")
print(f"The average sentiment score for the hateful comments is {hateful_sentiment}")
print(f"The average sentiment score for the non-hateful comments is {non_hateful_sentiment}")

print("######### Toxicity #########")
print(f"The average toxicity score for the hateful comments is {hateful_toxicity}")
print(f"The average toxicity score for the non-hateful comments is {non_hateful_toxicity}")


######### Sentiment #########
The average sentiment score for the hateful comments is -0.3437611764705882
The average sentiment score for the non-hateful comments is -0.04682222222222223
######### Toxicity #########
The average toxicity score for the hateful comments is 0.345255050238441
The average toxicity score for the non-hateful comments is 0.1513899951918511


You can also have more detailed information by calling the profile view's to_pandas method: 

In [6]:
hateful_profile.view().to_pandas()


,cardinality/est,cardinality/lower_1,cardinality/upper_1,counts/inf,counts/n,counts/nan,counts/null,distribution/max,distribution/mean,distribution/median,...,distribution/q_95,distribution/q_99,distribution/stddev,type,types/boolean,types/fractional,types/integral,types/object,types/string,types/tensor
column,,,,,,,,,,,,,,,,,,,,,
prompt,85.000018,85.0,85.004262,0,85,0,0,NaN,0.000000,NaN,...,NaN,NaN,0.000000,SummaryType.COLUMN,0,0,0,0,85,0
prompt.sentiment_nltk,65.000010,65.0,65.003256,0,85,0,0,0.863400,-0.343761,-0.504000,...,0.476700,0.863400,0.487009,SummaryType.COLUMN,0,85,0,0,0,0
prompt.toxicity,85.000018,85.0,85.004262,0,85,0,0,0.962283,0.345255,0.056012,...,0.952058,0.962283,0.418585,SummaryType.COLUMN,0,85,0,0,0,0
